In [1]:
import pandas as pd;
import numpy as np
import scipy.stats as st
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import psycopg2


In [2]:
def connection():
    con = psycopg2.connect(host='ec2-34-234-240-121.compute-1.amazonaws.com', database='d5q2q4m2lu6v8e',
    user='rtndvluihoudvx', password='21ed3821860243abaf3da0a620c45a963faa2e652af4ae42c81524e71efb8521')
    cur = con.cursor()
    sql = 'SELECT * FROM public.forms_responses;'
    cur.execute(sql)
    data = cur.fetchall()
    cols = []
    for elt in cur.description:
        cols.append(elt[0])

    df = pd.DataFrame(data=data,columns=cols)
    con.close()
    return df

In [6]:
data = pd.read_csv('dataset/data_update_2023.csv',index_col=[0])
#data = connection();
#data.to_csv('data_update_2023.csv')
data.info()
data.rename(columns={'age_group': 'Faixa de idade', 
                     'school_level': 'Nível escolar',
                    'religion': 'Denominação religiosa',
                     'vaccinated':'Vacinados',
                     'vaccine_doses':"Doses da Vacina"}, inplace=True)

#Ajustar dados
data.loc[data['Nível escolar'] == 'superior', 'Nível escolar'] = 'Superior';
data.loc[data['Nível escolar'] == 'médio', 'Nível escolar'] = 'Médio';
data.loc[data['Nível escolar'] == 'fundamental imcompleto', 'Nível escolar'] = 'Fundamental Incompleto';
data.loc[data['Nível escolar'] == 'fundamental', 'Nível escolar'] = 'Fundamental';
data.loc[data['Nível escolar'] == 'pós-graduação', 'Nível escolar'] = 'Pós-graduação';
data.to_csv('data_update_corrigido.csv')

 


<class 'pandas.core.frame.DataFrame'>
Int64Index: 5050 entries, 0 to 5049
Data columns (total 36 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   id                                 5050 non-null   object 
 1   age_group                          5050 non-null   object 
 2   school_level                       5050 non-null   object 
 3   religion                           5050 non-null   object 
 4   comorbidity                        5050 non-null   bool   
 5   diabetes                           5050 non-null   bool   
 6   heart_problem                      5050 non-null   bool   
 7   kidney_disease                     5050 non-null   bool   
 8   thyroid                            5050 non-null   bool   
 9   obesity                            5050 non-null   bool   
 10  other_comorbidity                  769 non-null    object 
 11  affected_covid_19                  5050 non-null   bool 

In [4]:
# Computa valores ausentes do dataset
def calculateMissingValues(nyc_data_raw):
        print("Detect missing values.")
        missing_values = nyc_data_raw.isnull().sum()
        percent_missing = ((missing_values / nyc_data_raw.index.size) * 100)
        print(percent_missing)
        

In [5]:
calculateMissingValues(data);


Detect missing values.
Unnamed: 0                             0.000000
id                                     0.000000
age_group                              0.000000
school_level                           0.000000
religion                               0.000000
comorbidity                            0.000000
diabetes                               0.000000
heart_problem                          0.000000
kidney_disease                         0.000000
thyroid                                0.000000
obesity                                0.000000
other_comorbidity                     84.772277
affected_covid_19                      0.000000
diagnostic_confirmation                0.000000
time_interval                         79.108911
diagnostic_method                     79.247525
treatment_place                       79.326733
hospital_treatment                    97.861386
covid_sequelae                        79.346535
vaccinated                             0.000000
vaccine_doses    

In [6]:
data.head()

,Unnamed: 0,id,age_group,school_level,religion,comorbidity,diabetes,heart_problem,kidney_disease,thyroid,...,covid_information,maintained_family_income,received_social_assistance,recovered_family_income,family_in_debt,created_at,updated_at,deleted_at,user_common_id,reason_not_cpf
0,0,175323bd-491b-4a4e-8d17-a957c556c0dd,51-70,fundamental imcompleto,Católica,True,True,False,False,False,...,Veículo televisivo e rádio,False,False,False,True,2022-09-22 20:16:40.153,2022-09-22 20:16:40.153,NaN,cb6a2cd8-f84b-4b3b-9702-ff92ac08a77a,NaN
1,1,797a69ef-abf1-4c7d-a955-33488f406e3a,até 25 anos,superior,Católica,False,False,False,False,False,...,Veículo televisivo e rádio,False,True,True,False,2022-09-28 14:02:14.044,2022-09-28 14:02:14.045,NaN,a20a0193-eeaf-404f-9ce3-51e66eeba9cb,NaN
2,2,c51d69e3-43dd-4fa3-a87b-cce4de4d7831,51-70,médio,Católica,False,False,False,False,False,...,Veículo televisivo e rádio,True,False,False,False,2022-10-04 22:15:57.037,2022-10-04 22:15:57.038,NaN,c8ee36f8-e206-4b4a-a896-5c24f9b332f2,NaN
3,3,6a368405-ebc9-4ac3-b5ac-3d2aa73c4638,até 25 anos,médio,Católica,False,False,False,False,False,...,Internet e redes sociais,True,True,False,False,2022-10-04 23:32:25.638,2022-10-04 23:32:25.639,NaN,4fd51acd-903a-4c55-a86c-f0ef850ac858,NaN
4,4,3e44b96f-db22-4e9f-ac91-60a776d744e7,até 25 anos,superior,Católica,False,False,False,False,False,...,Veículo televisivo e rádio,True,True,False,False,2022-10-05 11:06:35.447,2022-10-05 11:06:35.448,NaN,38d6def8-15d7-4c50-99c4-04bd52544b5b,NaN


In [62]:
fig = px.histogram(data, x="Faixa de idade",color="Faixa de idade",
                   category_orders={"Faixa de idade":['até 25 anos','25-35','36-50','51-70','acima de 70']})
fig.update_layout(legend_traceorder="reversed",
    legend_title="<b>Faixa de Idade</b>",title_x=0.5,
    yaxis_title_text='Quantidade', # yaxis label
   
)
fig.show()

In [63]:
fig = px.histogram(data, x="Nível escolar",title='Histograma do Nível Escolar dos Indivíduos',
                   color="Nível escolar",
                   category_orders={"Nível escolar":['fundamental incompleto','fundamental',
                                                     'médio','superior','pós-graduação']},
                   labels={'count':'Quantidade'})

fig.update_layout(
    legend_title="<b>Nível Escolar</b>",title_x=0.5,
    yaxis_title_text='Quantidade', # yaxis label
   
)
fig.update_traces(showlegend=True)
fig.show()

In [64]:
fig = px.histogram(data, x="Denominação religiosa",title='Histograma da Denominação religiosa',
                   color="Denominação religiosa",
                   category_orders={"Denominação religiosa":['Ateu','Católica','Evangélica',
                                                             'Religiões de Matrizes Afro-brasileira','Outro']},
                   labels={'count':'Quantidade'})

fig.update_layout(
    legend_title="<b>Religião</b>",title_x=0.5,
    yaxis_title_text='Quantidade', # yaxis label
   
)
fig.update_traces(showlegend=True)
fig.show()


In [65]:
#Agrupamentos
data_agrupamento_1 = data.groupby(['Faixa de idade',"Nível escolar"]).agg({"Nível escolar": ["count"]},
                                                                                 split_out=4)
data_agrupamento_1 = data_agrupamento_1.reset_index()
data_agrupamento_1.columns = ['Faixa de Idade','Escolaridade','Quantidade']
data_agrupamento_1.sort_values(by='Quantidade',ascending=False)

,Faixa de Idade,Escolaridade,Quantidade
2,25-35,Médio,578
7,36-50,Médio,548
11,51-70,Fundamental Incompleto,541
6,36-50,Fundamental Incompleto,459
29,até 25 anos,Médio,327
4,25-35,Superior,277
21,Até 25 anos,Médio,234
1,25-35,Fundamental Incompleto,215
5,36-50,Fundamental,213
9,36-50,Superior,195


In [66]:
fig = px.bar(data_agrupamento_1, x='Faixa de Idade', y="Quantidade",color="Escolaridade",
              
             labels={'Faixa de Idade':'<b>Faixa de Idade</b>','Escolaridade':'<b>Escolaridade</b>',},
             title="<b>Faixa de Idade por Escolaridade</b>")

fig.update_layout(
    legend_title="<b>Escolaridade</b>",title_x=0.5
   
)


In [67]:
#Agrupamentos
data_agrupamento_2 = data.groupby(["Nível escolar","Denominação religiosa"]).agg({"Denominação religiosa": ["count"]},
                                                                                 split_out=4)
data_agrupamento_2 = data_agrupamento_2.reset_index()
data_agrupamento_2.columns = ['Escolaridade','Religião','Quantidade']
data_agrupamento_2.sort_values(by='Quantidade',ascending=False)

,Escolaridade,Religião,Quantidade
11,Médio,Católica,1111
6,Fundamental Incompleto,Católica,951
12,Médio,Evangélica,621
7,Fundamental Incompleto,Evangélica,570
1,Fundamental,Católica,440
20,Superior,Católica,429
2,Fundamental,Evangélica,220
21,Superior,Evangélica,210
13,Médio,Outro,104
5,Fundamental Incompleto,Ateu,99


In [71]:
fig = px.bar(data_agrupamento_2, x='Religião', y="Quantidade",color="Escolaridade",
              
             labels={'Religião':'<b>Religião</b>','Escolaridade':'<b>Escolaridade</b>',},
             title="<b>Religião por Escolaridade</b>")

fig.update_layout(
    legend_title="<b>Escolaridade</b>",title_x=0.5
   
)

In [72]:
#Agrupamentos
data_agrupamento_3 = data.groupby(["Nível escolar","Vacinados"]).agg({"Vacinados": ["count"]},
                                                                                 split_out=4)
data_agrupamento_3 = data_agrupamento_3.reset_index()
data_agrupamento_3.columns = ['Escolaridade','Vacinados','Quantidade']
data_agrupamento_3.sort_values(by='Quantidade',ascending=False)

,Escolaridade,Vacinados,Quantidade
5,Médio,True,1851
3,Fundamental Incompleto,True,1622
8,Superior,True,708
1,Fundamental,True,695
6,Pós-graduação,True,68
2,Fundamental Incompleto,False,49
4,Médio,False,35
0,Fundamental,False,18
7,Superior,False,4


In [73]:
data_agrupamento_3_true = data_agrupamento_3[data_agrupamento_3.Vacinados]
fig = px.pie(data_agrupamento_3_true, values='Quantidade', names='Escolaridade', 
             title='Vacinados com Escolaridade')
fig.update_layout(
    legend_title="<b>Escolaridade</b>",title_x=0.5
   
)
fig.show()


In [74]:
data_agrupamento_3_false = data_agrupamento_3[~data_agrupamento_3.Vacinados]
fig = px.pie(data_agrupamento_3_false, values='Quantidade', names='Escolaridade', 
             title='Não Vacinados com Escolaridade')
fig.update_layout(
    legend_title="<b>Escolaridade</b>",title_x=0.5
   
)
fig.show()

In [75]:
#Agrupamentos
data_agrupamento_4 = data.groupby(["Denominação religiosa","Vacinados"]).agg({"Vacinados": ["count"]},
                                                                                 split_out=4)
data_agrupamento_4 = data_agrupamento_4.reset_index()
data_agrupamento_4.columns = ['Religião','Vacinados','Quantidade']
data_agrupamento_4.sort_values(by='Quantidade',ascending=False)

,Religião,Vacinados,Quantidade
3,Católica,True,2941
5,Evangélica,True,1577
7,Outro,True,227
1,Ateu,True,177
4,Evangélica,False,64
2,Católica,False,29
9,Religiões de Matrizes Afro-brasileira,True,22
6,Outro,False,7
0,Ateu,False,4
8,Religiões de Matrizes Afro-brasileira,False,2


In [76]:
data_agrupamento_4_true = data_agrupamento_4[data_agrupamento_4.Vacinados]
fig = px.pie(data_agrupamento_4_true, values='Quantidade', names='Religião', 
             title='Vacinados por Religião')
fig.update_layout(
    legend_title="<b>Religião</b>",title_x=0.5
   
)
fig.show()


In [77]:
data_agrupamento_4_false = data_agrupamento_4[~data_agrupamento_4.Vacinados]
fig = px.pie(data_agrupamento_4_false, values='Quantidade', names='Religião', 
             title='Não Vacinados por Religião')
fig.update_layout(
    legend_title="<b>Religião</b>",title_x=0.5
   
)
fig.show()


In [11]:
matrix = data.corr()
matrix


,comorbidity,diabetes,heart_problem,kidney_disease,thyroid,obesity,affected_covid_19,diagnostic_confirmation,Vacinados,lost_family_member,affected_covid_after_vaccinated,rehabilitation_sequelae,maintained_family_income,received_social_assistance,recovered_family_income,family_in_debt,deleted_at
comorbidity,1.000000,0.430554,0.284726,0.173136,0.122170,0.096046,0.077074,0.082830,0.017290,0.019845,0.052017,0.079265,0.017721,-0.147850,-0.033514,0.007065,NaN
diabetes,0.430554,1.000000,0.096127,0.039570,0.056347,-0.011570,0.013468,0.021628,0.007826,-0.012542,0.008953,0.027183,-0.015029,-0.084185,0.008548,0.030910,NaN
heart_problem,0.284726,0.096127,1.000000,0.045434,0.073851,-0.007651,0.031744,0.075960,0.012672,0.001516,0.061659,0.073384,-0.030298,-0.060767,-0.006039,0.027105,NaN
kidney_disease,0.173136,0.039570,0.045434,1.000000,0.095732,-0.004652,0.025309,0.034071,-0.017012,0.012617,0.022232,0.063143,-0.013975,0.002047,-0.000016,0.021356,NaN
thyroid,0.122170,0.056347,0.073851,0.095732,1.000000,-0.003283,0.030188,0.058247,0.009462,0.016238,0.015688,-0.007717,-0.009861,-0.014508,0.004116,0.021948,NaN
obesity,0.096046,-0.011570,-0.007651,-0.004652,-0.003283,1.000000,0.032307,0.012695,0.007439,0.000345,0.029005,0.027125,-0.014207,0.014449,0.008728,0.029735,NaN
affected_covid_19,0.077074,0.013468,0.031744,0.025309,0.030188,0.032307,1.000000,0.398393,0.049932,0.071351,0.252056,0.102635,-0.046371,-0.016988,0.044512,0.053552,NaN
diagnostic_confirmation,0.082830,0.021628,0.075960,0.034071,0.058247,0.012695,0.398393,1.000000,0.033757,0.094777,0.257153,0.126037,0.009626,-0.054023,0.022438,0.005983,NaN
Vacinados,0.017290,0.007826,0.012672,-0.017012,0.009462,0.007439,0.049932,0.033757,1.000000,0.030162,0.032223,-0.005980,0.012549,-0.008592,0.013615,-0.008266,NaN
lost_family_member,0.019845,-0.012542,0.001516,0.012617,0.016238,0.000345,0.071351,0.094777,0.030162,1.000000,0.037756,0.039363,0.013478,-0.012258,-0.006217,-0.007814,NaN


In [15]:
mask = np.triu(np.ones_like(matrix, dtype=bool))
rLT = matrix.mask(mask)

heat = go.Heatmap(
    z = rLT,
    x = rLT.columns.values,
    y = rLT.columns.values,
    zmin = - 0.25,
    zmax = 1,
    xgap = 1, 
    ygap = 1,
    colorscale = px.colors.diverging.RdBu
)

title = 'Matrix de Correlação'

layout = go.Layout(
    title_text=title, 
    title_x=0.5, 
    width=600, 
    height=600,
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    yaxis_autorange='reversed'
)

fig=go.Figure(data=[heat], layout=layout)
fig.show()

In [19]:
fig = px.bar(data, x='Doses da Vacina',
              
             labels={'count':'<b>Quantidade</b>','Doses da Vacina':'<b>Doses da Vacina</b>',},
             title="<b>Sumarização das amostras por doses da vacina</b>")

fig.update_layout(
    title_x=0.5
   
)
fig.show()
